In [ ]:
import pickle

with open('/content/BARTScore/SUM/Newsroom/data.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
!git clone https://github.com/neulab/BARTScore.git

fatal: destination path 'BARTScore' already exists and is not an empty directory.


In [ ]:
data.keys()

dict_keys(['2140', '7569', '30385', '2591', '1943', '8167', '10113', '7651', '2350', '31823', '9821', '30429', '7208', '10376', '7666', '9823', '30821', '9092', '10506', '32563', '6476', '6130', '8727', '10580', '34005', '8801', '5678', '5626', '10488', '144', '31425', '9912', '10623', '30117', '31059', '6578', '31788', '10062', '8607', '1807', '32146', '3829', '32202', '6726', '9642', '4003', '5926', '6638', '32716', '32670', '350', '8212', '6906', '33233', '6314', '9950', '34184', '171', '7118', '1295'])

In [ ]:
data['2140'].keys()

dict_keys(['src', 'ref_summ', 'sys_summs'])

In [ ]:
data['2140']['src']

"A worker sets up a polling station the morning of the GOP primary in Florida . Fewer voters than expected turned out . Editor 's note : John Avlon is a CNN contributor and senior political columnist for Newsweek and The Daily Beast . He is co-editor of the book `` Deadline Artists : America 's Greatest Newspaper Columns . '' ( CNN ) -- Beneath Rick Santorum 's stunning three-state sweep on Tuesday stands another stubborn sign of dissatisfaction with the status quo : Republican turnout is down . I 'm talking embarrassingly , disturbingly , hey-don't-you-know-it's-an-election-year bad . It is a sign of a serious enthusiasm gap among the rank and file , and a particularly bad omen for Mitt Romney and the GOP in the general election . Here 's the tale of the tape , state by state , beginning with Tuesday night : Minnesota had just more than 47,000 people turn out for its caucuses this year -- four years ago it was nearly 63,000 -- and Romney came in first , not a distant third as he did T

In [ ]:
data['2140']['ref_summ']

'John Avlon says low voter turnout in the primaries is a sign of a serious enthusiasm gap among the rank and file , a bad omen for the GOP .'

In [ ]:
data['2140']['sys_summs'].keys()

dict_keys(['fragments', 'textrank', 'abstractive', 'pointer_c', 'pointer_n', 'pointer_s', 'lede3'])

In [ ]:
data['2140']['sys_summs']['textrank'].keys()

dict_keys(['sys_summ', 'scores'])

In [ ]:
data['2140']['sys_summs']['textrank']['sys_summ']

'In New Hampshire , the same dynamic applied -- 245,000 voters turned out in 2012 , compared with 241,000 four years before , despite Republicans being the only game in town and independents making up 47 % of the total turnout in 2012 , according to CNN exit polls .'

In [ ]:
data['2140']['sys_summs']['textrank']['scores']

{'coherence': 4.0,
 'fluency': 4.0,
 'informativeness': 3.3333333333333335,
 'relevance': 4.0}

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

In [ ]:
class CustomBartScorer:
    def __init__(self, model_name="facebook/bart-large-cnn", device="cuda"):
        """
        Initialize the tokenizer and model for computing BartScore.
        Args:
            model_name (str): Pretrained BART model checkpoint.
            device (str): Device to run computations on ('cuda' or 'cpu').
        """
        self.device = device
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name)
        self.model.to(device)
        self.model.eval()

    def compute_log_probs(self, src_text, tgt_text):
        """
        Compute the log probabilities of the target text given the source text.
        Args:
            src_text (str): Source text (e.g., source document).
            tgt_text (str): Target text (e.g., system summary or reference summary).
        Returns:
            log_prob (float): The log probability of the target text.
        """
        # Tokenize source and target texts
        src_inputs = self.tokenizer(src_text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(self.device)
        tgt_inputs = self.tokenizer(tgt_text, return_tensors="pt", max_length=1024, truncation=True, padding=True).to(self.device)

        # Forward pass with source as input and target as labels
        with torch.no_grad():
            outputs = self.model(**src_inputs, labels=tgt_inputs["input_ids"])
            logits = outputs.logits  # Logits: (batch_size, seq_len, vocab_size)

        # Compute log probabilities using log-softmax
        log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

        # Gather log probabilities of the target tokens
        tgt_token_ids = tgt_inputs["input_ids"]
        tgt_mask = tgt_inputs["attention_mask"]
        seq_len = tgt_mask.sum(dim=1)

        # Collect log probabilities for the correct target tokens
        tgt_log_probs = log_probs.gather(2, tgt_token_ids.unsqueeze(-1)).squeeze(-1)

        # Mask out padding tokens and sum log probabilities
        tgt_log_probs = tgt_log_probs * tgt_mask
        total_log_probs = tgt_log_probs.sum(dim=1)

        # Normalize by sequence length
        normalized_log_probs = total_log_probs / seq_len

        return normalized_log_probs.item()

    def compute_bartscore(self, src, tgt):
        """
        Compute BartScore for a given source and target text.
        Args:
            src (str): Source text.
            tgt (str): Target text.
        Returns:
            score (float): BartScore value.
        """
        return self.compute_log_probs(src, tgt)


In [ ]:
scorer = CustomBartScorer(model_name="facebook/bart-large-cnn", device="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
source = data['2140']["src"]
ref_summary = data['2140']["ref_summ"]
sys_summary = data['2140']["sys_summs"]['textrank']['sys_summ']
bart_score = scorer.compute_bartscore(sys_summary, ref_summary)
print(f"BartScore: {bart_score}")

BartScore: -3.3545756340026855


In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

import evaluate
from nltk.tokenize import sent_tokenize

rouge = evaluate.load("rouge")

In [ ]:
# Store ROUGE-1 and BartScore results
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
bart_scores = []

for element in data.values():
    src = element["src"]
    ref_summ = element["ref_summ"]
    sys_summary = element["sys_summs"]

    # For each system summary variant (fragments, textrank, etc.), compute ROUGE and BartScore
    for key, sys_sum_dict in sys_summary.items():
        sys_sum = sys_sum_dict["sys_summ"]
        scores = sys_sum_dict["scores"]

        # Compute ROUGE score
        rouge_result = rouge.compute(predictions=[sys_sum], references=[ref_summ])

        # Store Rouge1 / Rouge2 / RougeL scores
        rouge1_score = rouge_result['rouge1']
        rouge2_score = rouge_result['rouge2']
        rougeL_score = rouge_result['rougeL']

        # Compute BartScore
        bart_score = scorer.compute_bartscore(src, sys_sum)

        # Append the results
        rouge1_scores.append(rouge1_score)
        rouge2_scores.append(rouge2_score)
        rougeL_scores.append(rougeL_score)
        bart_scores.append(bart_score)



In [ ]:
coh = []
flu = []
info = []
rel = []

for element in data.values():
    src = element["src"]
    ref_summ = element["ref_summ"]
    sys_summary = element["sys_summs"]

    # For each system summary variant (fragments, textrank, etc.), compute ROUGE-1 and BartScore
    for key, sys_sum_dict in sys_summary.items():
        coh.append(sys_sum_dict['scores']["coherence"])
        flu.append(sys_sum_dict['scores']["fluency"])
        info.append(sys_sum_dict['scores']["informativeness"])
        rel.append(sys_sum_dict['scores']["relevance"])

In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(rouge1_scores, coh)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.08937495069583032
P-value: 0.06727295916026399


In [ ]:
# Calculate Spearman correlation between BartScore and COH
corr, p_value = spearmanr(bart_scores, coh)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.6247454492952434
P-value: 7.621659394815542e-47


In [ ]:
!pip install moverscore

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [ ]:
!pip install pyemd
!pip install pytorch_pretrained_bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.9 MB/s eta 0:00:00


In [ ]:
from moverscore import word_mover_score, get_idf_dict
import numpy as np

[--------------------------------------------------]


/usr/local/lib/python3.10/dist-packages/pytorch_pretrained_bert/modeling.py:603: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_loca

In [ ]:
# Initialize lists to store generated summaries and reference summaries
generated_texts = []
reference_texts = []

# Iterate through the dataset to extract generated and reference texts
for element in data.values():
    ref_summ = element['ref_summ']  # Reference summary
    sys_summaries = element['sys_summs']  # System-generated summaries collection

    for sys_name, sys_data in sys_summaries.items():
        sys_summ = sys_data['sys_summ']  # Extract system-generated summary

        # Add generated and reference texts to the respective lists
        generated_texts.append(sys_summ)
        reference_texts.append(ref_summ)


In [ ]:
# Calculate the IDF for reference and generated summaries
idf_reference = get_idf_dict(reference_texts)
idf_generated = get_idf_dict(generated_texts)

# Calculate MoverScore
mover_scores = word_mover_score(
    reference_texts,          # List of reference summaries
    generated_texts,          # List of generated summaries
    idf_reference,            # IDF dictionary for reference texts
    idf_generated,            # IDF dictionary for generated texts
    stop_words=[],            # Stopwords, typically used to remove non-essential words
    n_gram=1,                 # Use n-gram, default is 1 (unigram)
    remove_subwords=True,     # Whether to remove subwords
    batch_size=8,             # Batch size, adjust to improve calculation speed
    device='cuda'             # Choose computing device, e.g., 'cuda' or 'cpu'
)

In [ ]:
print(f"MoverScore: {mover_scores}")

MoverScore: [0.6443659943650033, -0.0975651378471547, -0.1674827449099776, -0.06380617310423342, -0.022069119871935605, -0.026302616397309464, -0.022481765900183026, 0.027062012660817092, 0.8582724117840813, 0.24099229829800684, -0.08201828437115455, -0.14490503358300888, -0.020782506203897544, 0.1173039639128558, -0.048402421317363054, 0.5252036284162317, -0.05651812396520017, -0.22360462486757404, -0.025139835822427514, -0.015850034516798894, -0.09210653280002279, -0.17566555315860444, -0.03660430725619679, -0.08502075188982294, -0.14731689691116512, 0.28627290240799996, -0.20522721183164427, -0.21260498793138027, -0.11135289014875394, 0.15498794496845314, -0.13670811160668084, -0.12676933656232303, -0.1280275397616546, -0.10867235075851789, -0.1261064357303463, -0.07536147865991438, 0.37193658346650926, -0.07228040097024402, -0.17619373371331304, -0.022718053956901407, -0.17363561773568903, -0.11525647323524457, 0.4563836034336717, 0.9254228234068088, 0.20596120292667086, -0.3156646

In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(mover_scores, coh)

print(f"Spearman correlation of mover: {corr}")
print(f"P-value of mover: {p_value}")

Spearman correlation of mover: 0.16533263993512104
P-value of mover: 0.0006698000474395901


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00


In [ ]:
from bert_score import BERTScorer

In [ ]:
# initialize BERTScorer
scorer = BERTScorer(lang="en", rescale_with_baseline=True)

# calculate BERTScore
P, R, F = scorer.score(generated_texts, reference_texts)

# print BERTScore
for i in range(len(generated_texts)):
    print(f"BERTScore: {P[i].item():.4f}, Recall: {R[i].item():.4f}, F1-Score: {F[i].item():.4f}\n")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: 0.7077, Recall: 0.6969, F1-Score: 0.7028

BERTScore: 0.0029, Recall: 0.0927, F1-Score: 0.0490

BERTScore: -0.2030, Recall: -0.1084, F1-Score: -0.1543

BERTScore: -0.0119, Recall: 0.1226, F1-Score: 0.0560

BERTScore: 0.1821, Recall: 0.1737, F1-Score: 0.1792

BERTScore: 0.1694, Recall: 0.1703, F1-Score: 0.1712

BERTScore: 0.0658, Recall: 0.2171, F1-Score: 0.1418

BERTScore: -0.0520, Recall: 0.6743, F1-Score: 0.2871

BERTScore: 0.8872, Recall: 0.8756, F1-Score: 0.8816

BERTScore: 0.2331, Recall: 0.7442, F1-Score: 0.4775

BERTScore: -0.1610, Recall: -0.0968, F1-Score: -0.1273

BERTScore: -0.2724, Recall: -0.0436, F1-Score: -0.1589

BERTScore: 0.1216, Recall: 0.1461, F1-Score: 0.1352

BERTScore: 0.0286, Recall: 0.4039, F1-Score: 0.2107

BERTScore: -0.1326, Recall: 0.2023, F1-Score: 0.0308

BERTScore: 0.7121, Recall: 0.6528, F1-Score: 0.6828

BERTScore: 0.0004, Recall: 0.0637, F1-Score: 0.0334

BERTScore: 0.0727, Recall: -0.0306, F1-Score: 0.0221

BERTScore: -0.0675, Recall: 0.187

In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(F, coh)

print(f"Spearman correlation of BERTScore: {corr}")
print(f"P-value of BERTScore: {p_value}")

Spearman correlation of BERTScore: 0.1694246568287391
P-value of BERTScore: 0.0004883511314455359


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(F, flu)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.15406962938632385
P-value: 0.0015401641155744902


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(mover_scores, flu)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.10763894433819626
P-value: 0.02739942574786073


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(F, info)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.19615354758472714
P-value: 5.1799283857121995e-05


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(mover_scores,info)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.22599638693025603
P-value: 2.8911047936252053e-06


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(F, rel)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.17634863442108228
P-value: 0.00028144795501462147


In [ ]:
from scipy.stats import spearmanr

# Calculate Spearman correlation between ROUGE-1 and COH
corr, p_value = spearmanr(mover_scores, info)

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

Spearman correlation: 0.22599638693025603
P-value: 2.8911047936252053e-06
